# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
N. Pulatova  ->  N. Pulatova  |  ['N. Pulatova']
A. Gould  ->  A. Gould  |  ['A. Gould']
J. Li  ->  J. Li  |  ['J. Li']


J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']
Arxiv has 93 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2312.00924


extracting tarball to tmp_2312.00924...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figures/CGr35_moffat_100Angwide
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure figures/CGr35_moffat_100Angwide as tmp_2312.00924/./figures/CGr35_moffat_100Angwide.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


...

✔ → 99665:\section{Model fitting}
  ↳ 101480:\section{Extraction of the total intensity SPF from IRDIS data}
✔ → 101480:\section{Extraction of the total intensity SPF from IRDIS data}
  ↳ 102631:\section{Incompatibility of the silicate, amorphous carbon and water ice porous particle model}


✔ → 102631:\section{Incompatibility of the silicate, amorphous carbon and water ice porous particle model}
  ↳ 110323:end


J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']


Found 112 bibliographic references in tmp_2312.02000/HD181327_v3.bbl.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Pawellek} {et~al.}(2021){Pawellek}, {Wyatt}, {Matr{à}}, {Kennedy},   {Yelverton6}}]{Pawellek2021}{Pawellek}, N., {Wyatt}, M., {Matr{à}}, L., {Kennedy}, G.,  {Yelverton6},  B. 2021, \mnras, 502, 5390
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.00924-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.00924) | **MAGIC: Muse gAlaxy Groups In Cosmos -- A survey to probe the impact of  environment on galaxy evolution over the last 8 Gyr**  |
|| B. Epinat, et al. -- incl., <mark>L. Boogaard</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *27 pages, 22 figures, accepted for publication in A&A*|
|**Abstract**| We introduce the MUSE gAlaxy Groups in COSMOS (MAGIC) survey built to study the impact of environment on galaxy evolution over the last 8 Gyr. It consists of 17 MUSE fields targeting 14 massive structures at intermediate redshift ($0.3<z<0.8$) in the COSMOS area. We securely measure redshifts for 1419 sources and identify 76 galaxy pairs and 67 groups of at least 3 members using a friends-of-friends algorithm. The environment of galaxies is quantified from group properties, as well as from global and local density estimators, inferred from galaxy number density and dynamics within groups. The MAGIC survey increases the number of objects with a secure spectroscopic redshift over its footprint by a factor of $\sim 5$. Most of the new redshifts have apparent magnitudes in the $z^{++}$ band $z_{app}^{++}>21.5$. The spectroscopic redshift completeness is high: in the redshift range of [OII] emitters ($0.25 \le z < 1.5$), where most of the groups are found, it globally reaches a maximum of 80% down to $z_{app}^{++}=25.9$, and it locally decreases from $\sim 100$% to $\sim50$% in magnitude bins from $z_{app}^{++}=23-24$ to $z_{app}^{++}=25.5$. We find that the fraction of quiescent galaxies increases with local density and with the time spent in groups. A morphological dichotomy is also found between bulge-dominated quiescent and disk-dominated star-forming galaxies. As environment gets denser, the peak of the stellar mass distribution shifts towards $M_*>10^{10} M_\odot, the fraction of galaxies with $M_*<10^9 M_\odot$ decreasing significantly, even for star-forming galaxies. We also highlight peculiar features such as close groups, extended nebulae, and a gravitational arc. Our results suggest that galaxies are pre-processed in groups of increasing mass before entering rich groups and clusters. We publicly release two catalogs containing the properties of galaxies and groups, respectively. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.01174-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.01174) | **Separating the spectral counterparts in NGC 1275/Perseus cluster in  X-rays**  |
|| E. Fedorova, et al. -- incl., <mark>N. Pulatova</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *18 pages, 6 pages, 8 tables*|
|**Abstract**| We develop the recipe to separate the spectral counterparts of the AGN NGC 1275 from the emission of the Perseus cluster surrounding it in the spectra observed by Suzaku/XIS cameras with no usage of the spectral fitting models. The Perseus cluster emission reaches higher energies than is typical for the most AGN-situated dense surroundings (i.e. up to 9-10 keV). That is why the separation between the AGN and cluster spectra is especially important in this case. To avoid the degeneracy due to the huge quantity of the spectral fitting parameters such as abundances of elements the cluster consists of, thermal and Compton emission of the nucleus itself, and the jet SSC/IC emission spectral parameters as well we prefer to avoid the spectral fitting usage to perform this task. Instead, we use the spatial resolution of the components and double background subtracting. For this purpose we choose the following regions to collect all the photons from them: (1) circular or square-shaped region around the source (AGN); (2) ring-shaped (or non-overlapped square) region surrounding the AGN (for cluster); (3) remote empty circular region for the background. Having collected the photons from those regions we subtract the background (i.e. photons from the third region) from the source and cluster spectra. Next, we subtract the re-normalized cluster counts from the AGN spectrum; using the relation between the emission line amplitudes in the AGN and cluster spectra as the renormalization coefficient. We have performed this procedure on the whole set of the Suzaku/XIS observational data for NGC 1275 to obtain the cleaned spectra and light curve of the AGN emission in this system. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.02000-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.02000) | **The polarisation properties of the HD 181327 debris ring. Evidence for  sub-micron particles from scattered light observations**  |
|| J. Milli, et al. -- incl., <mark>J. Olofsson</mark>, <mark>C. Desgrange</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *21 pages, 22 figures, accepted in Astronomy & Astrophysics*|
|**Abstract**| Polarisation is a powerful remote-sensing tool to study the nature of particles scattering the starlight. It is widely used to characterise interplanetary dust particles in the Solar System and increasingly employed to investigate extrasolar dust in debris discs' systems. We aim to measure the scattering properties of the dust from the debris ring around HD 181327 at near-infrared wavelengths. We obtained high-contrast polarimetric images of HD 181327 in the H band with the SPHERE / IRDIS instrument on the Very Large Telescope (ESO). We complemented them with archival data from HST / NICMOS in the F110W filter reprocessed in the context of the Archival Legacy Investigations of Circumstellar Environments (ALICE) project. We developed a combined forward-modelling framework to simultaneously retrieve the scattering phase function in polarisation and intensity. We detected the debris disc around HD 181327 in polarised light and total intensity. We measured the scattering phase function and the degree of linear polarisation of the dust at 1.6 micron in the birth ring. The maximum polarisation is 23.6% +/- 2.6% and occurs between a scattering angle of 70 deg and 82 deg. We show that compact spherical particles made of a highly refractive and relatively absorbing material in a differential power-law size distribution of exponent $-3.5$ can simultaneously reproduce the polarimetric and total intensity scattering properties of the dust. This type of material cannot be obtained with a mixture of silicates, amorphous carbon, water ice, and porosity, and requires a more refracting component such as iron-bearing minerals. We reveal a striking analogy between the near-infrared polarisation of comets and that of HD 181327. The methodology developed here combining VLT/SPHERE and HST/NICMOS may be applicable in the future to combine the polarimetric capabilities of SPHERE with the sensitivity of JWST. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.01122-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.01122) | **GeV Variability Properties of TeV Blazars Detected by Fermi-LAT**  |
|| G. Wang, H. Xiao, J. Fan, <mark>X. Zhang</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *22 pages, 9 figures, accepted for publication in ApJS*|
|**Abstract**| Variability is a prominent observational feature of blazars. The high-energy radiation mechanism of jets has always been important but still unclear. In this work, we performed a detailed analysis using Fermi-LAT data across 15 years and obtained GeV light curve information for 78 TeV blazars detected by Fermi. We provided annual GeV fluxes and corresponding spectral indices for the 78 TeV blazars and thorough monthly GeV fluxes for a subsample of 41 bright blazars. Our results suggest a strong correlation between the ${\gamma}$-ray photon index and $\log L_{\rm \gamma}$ for the flat spectrum radio quasars (FSRQs) and high-energy-peaked BL Lacs (HBLs). 14 sources in our sample show significant GeV outbursts/flares above the relatively stable, low-flux light curve, with 6 of them showing a clear sharp peak profile in their 5-day binned light curves. We quantified the variability utilizing the fractional variability parameter $F_{\mathrm{var}}$, and found that the flux of the FSRQs showed significantly stronger variability than that of the BL Lacs. The 41 bright blazars in this work are best fit by a log-normal flux distribution. We checked the spectral behavior and found 11 out of the 14 sources show a 'bluer-when-brighter (BWB)' trend, suggesting this spectral behavior for these TeV blazars at the GeV band arises from the mechanism that the synchrotron-self Compton (SSC) process dominates the GeV emission. Our research offers a systematic analysis of the GeV variability properties of TeV blazars and serves as a helpful resource for further associated blazar studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.01691-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.01691) | **Estimating Coronal Mass Ejection Mass and Kinetic Energy by Fusion of  Multiple Deep-learning Models**  |
|| K. A. Alobaid, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *10 pages, 7 figures*|
|**Abstract**| Coronal mass ejections (CMEs) are massive solar eruptions, which have a significant impact on Earth. In this paper, we propose a new method, called DeepCME, to estimate two properties of CMEs, namely, CME mass and kinetic energy. Being able to estimate these properties helps better understand CME dynamics. Our study is based on the CME catalog maintained at the Coordinated Data Analysis Workshops (CDAW) Data Center, which contains all CMEs manually identified since 1996 using the Large Angle and Spectrometric Coronagraph (LASCO) on board the Solar and Heliospheric Observatory (SOHO). We use LASCO C2 data in the period between January 1996 and December 2020 to train, validate and test DeepCME through 10-fold cross validation. The DeepCME method is a fusion of three deep learning models, including ResNet, InceptionNet, and InceptionResNet. Our fusion model extracts features from LASCO C2 images, effectively combining the learning capabilities of the three component models to jointly estimate the mass and kinetic energy of CMEs. Experimental results show that the fusion model yields a mean relative error (MRE) of 0.013 (0.009, respectively) compared to the MRE of 0.019 (0.017, respectively) of the best component model InceptionResNet (InceptionNet, respectively) in estimating the CME mass (kinetic energy, respectively). To our knowledge, this is the first time that deep learning has been used for CME mass and kinetic energy estimations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.01278-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.01278) | **OGLE-2019-BLG-1180Lb: Discovery of a Wide-orbit Jupiter-mass Planet  around a Late-type Star**  |
|| S.-J. Chung, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-12-05*|
|*Comments*| *12 pages, 12 figures, 5 tables, published in AJ*|
|**Abstract**| We report on the discovery and analysis of the planetary microlensing event OGLE-2019-BLG-1180 with a planet-to-star mass ratio $q \sim 0.003$. The event OGLE-2019-BLG-1180 has unambiguous cusp-passing and caustic-crossing anomalies, which were caused by a wide planetary caustic with $s \simeq 2$, where $s$ is the star-planet separation in units of the angular Einstein radius $\theta_{E}$. Thanks to well-covered anomalies by the Korea Micorolensing Telescope Network (KMTNet), we measure both the angular Einstein radius and the microlens parallax in spite of a relatively short event timescale of $t_{E} = 28$ days. However, because of a weak constraint on the parallax, we conduct a Bayesian analysis to estimate the physical lens parameters. We find that the lens system is a super-Jupiter-mass planet of $M_{p} = 1.75^{+0.54}_{-0.51} M_{J}$ orbiting a late-type star of $M_{h}=0.55^{+0.27}_{-0.26} M_\odot$ at a distance of $D_{L} = 6.1^{+0.9}_{-1.3}$ kpc. The projected star-planet separation is $a_{\perp} = 5.19^{+0.90}_{-1.23}$ au, which means that the planet orbits at about four times the snow line of the host star. Considering the relative lens-source proper motion of $\mu_{rel} = 6$ mas/yr, the lens will be separated from the source by 60 mas in 2029. At that time one can measure the lens flux from adaptive optics imaging of Kec or a next-generation 30 m class telescope. OGLE-2019-BLG-1180Lb represents a growing population of wide-orbit planets detected by KMTNet, so we also present a general investigation into prospects for further expanding the sample of such planets. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2312.00924.md
    + _build/html/tmp_2312.00924/./figures/histo_mstar_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png
    + _build/html/tmp_2312.00924/./figures/histo_BD_ReffG_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png
    + _build/html/tmp_2312.00924/./figures/histo_NUV-Rp_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png
    + _build/html/tmp_2312.00924/./figures/histo_sfr_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png
    + _build/html/tmp_2312.00924/./figures/arc_CGr32_map.png
    + _build/html/tmp_2312.00924/./figures/histo_mstar_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover_nored.png
    + _build/html/tmp_2312.00924/./figures/histo_BD_ReffG_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover_nored.png
exported in  _build/html/2312.01174.md
    + _build/html/tmp_2312.01174/./Screenshot_20231107_130040.png
    + _build/html/tmp_2312.01174/./Screenshot_20231107_130713.png
    + _build/html/tmp_2312.01174/./lightcurve.png
    + _build/html/tmp_2312.01174/./spectral_

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nii}{[\ion{N}{ii}]}$
$\newcommand{\niia}{[\ion{N}{ii}]\lambda6548}$
$\newcommand{\niib}{[\ion{N}{ii}]\lambda6583}$
$\newcommand{\niiab}{[\ion{N}{ii}]\lambda\lambda6548,6583}$
$\newcommand{\sii}{[\ion{S}{ii}]}$
$\newcommand{\siia}{[\ion{S}{ii}]\lambda6716}$
$\newcommand{\siib}{[\ion{S}{ii}]\lambda6731}$
$\newcommand{\siiab}{[\ion{S}{ii}]\lambda\lambda6716,6731}$
$\newcommand{\oii}{[\ion{O}{ii}]}$
$\newcommand{\oiiab}{[\ion{O}{ii}]\lambda\lambda3727,3729}$
$\newcommand{\oiia}{[\ion{O}{ii}]\lambda3727}$
$\newcommand{\oiib}{[\ion{O}{ii}]\lambda3729}$
$\newcommand{\oiii}{[\ion{O}{iii}]}$
$\newcommand{\oiiiab}{[\ion{O}{iii}]\lambda\lambda4959,5007}$
$\newcommand{\oiiia}{[\ion{O}{iii}]\lambda5007}$
$\newcommand{\oiiib}{[\ion{O}{iii}]\lambda4959}$
$\newcommand{\oiiic}{[\ion{O}{iii}]\lambda4363}$
$\newcommand{\ciii}{\ion{C}{iii}]}$
$\newcommand{\ciiiab}{\ion{C}{iii}]\lambda\lambda1907,1909}$
$\newcommand{\ciiia}{\ion{C}{iii}]\lambda1907}$
$\newcommand{\ciiib}{\ion{C}{iii}]\lambda1909}$
$\newcommand{\ca}{[\ion{Ca}{iii}]}$
$\newcommand{\mgiiab}{\ion{Mg}{ii}\lambda\lambda2796,2803}$
$\newcommand{\mgii}{\ion{Mg}{ii}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\haa}{H\alpha\lambda6563}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hba}{H\beta\lambda4861}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyaa}{Ly\alpha\lambda1216}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\zCOSMOS}{COSMOS-z_{\rm spec}}$</div>



<div id="title">

# MAGIC: Muse gAlaxy Groups In Cosmos - A survey to probe the impact of environment on galaxy evolution over the last 8 Gyr   $\thanks{Based on observations made with ESO telescopes at the Paranal Observatory under programs 094.A-0247, 095.A-0118, 096.A-0596, 097.A-0254, 099.A-0246, 100.A-0607, 101.A-0282, 102.A-0327, 103.A-0563.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.00924-b31b1b.svg)](https://arxiv.org/abs/2312.00924)<mark>Appeared on: 2023-12-05</mark> -  _27 pages, 22 figures, accepted for publication in A&A_

</div>
<div id="authors">

B. Epinat, et al. -- incl., <mark>L. Boogaard</mark>

</div>
<div id="abstract">

**Abstract:** Galaxies migrate along cosmic web filaments from small groups to clusters, which makes the evolution of their properties look stronger as environments get denser. We introduce the Muse gAlaxy Groups in Cosmos (MAGIC) survey built to study the impact of environment on galaxy evolution down to low stellar masses over the last eight Gyr. The MAGIC survey consists of 17 Multi-Unit Spectrocopic Exporer (MUSE) fields targeting 14 massive, known structures at intermediate redshift ( $0.3<z<0.8$ ) in the COSMOS area, with a total on-source exposure of 67h. We securely measure redshifts for 1419 sources and identify 76 galaxy pairs and 67 groups of at least three members using a friends-of-friends algorithm.   The environment of galaxies is quantified from group properties, as well as from global and local density estimators, inferred from galaxy number density and dynamics within groups. The MAGIC survey has increased the number of objects with a secure spectroscopic redshift over its footprint by a factor of $\sim 5$ compared to the previous extensive spectroscopic campaigns on the COSMOS field. Most of the new redshifts have apparent magnitudes in the $z^{++}$ band $z_{\rm app}^{++}>21.5$ . The spectroscopic redshift completeness is high: in the redshift range of $\oii$ emitters ( $0.25 \le z < 1.5$ ), where most of the groups are found, it globally reaches a maximum of 80 \% down to $z_{\rm app}^{++}=25.9$ , and it locally decreases from $\sim 100$ \% to $\sim50$ \% in magnitude bins from $z_{\rm app}^{++}=23-24$ to $z_{\rm app}^{++}=25.5$ .   We find that the fraction of quiescent galaxies increases with local density and with the time spent in groups. A morphological dichotomy is also found between bulge-dominated quiescent and disk-dominated star-forming galaxies. As environment gets denser, the peak of the stellar mass distribution shifts towards $M_\star>10^{10} M_\sun$ , the fraction of galaxies with $M_\star<10^9 M_\sun$ decreasing significantly, even for star-forming galaxies.   We also highlight peculiar features such as close groups, extended nebulae, and a gravitational arc. Our results suggest that galaxies are pre-processed in groups of increasing mass before entering rich groups and clusters.   We publicly release two catalogs containing the properties of galaxies and groups, respectively.

</div>

<div id="div_fig1">

<img src="tmp_2312.00924/./figures/histo_mstar_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png" alt="Fig21.1" width="25%"/><img src="tmp_2312.00924/./figures/histo_BD_ReffG_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png" alt="Fig21.2" width="25%"/><img src="tmp_2312.00924/./figures/histo_NUV-Rp_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png" alt="Fig21.3" width="25%"/><img src="tmp_2312.00924/./figures/histo_sfr_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover.png" alt="Fig21.4" width="25%"/>

**Figure 21. -** Histograms of stellar mass (top left panel), bulge-to-disc ratio within the galaxy effective radius (top right panel), color (bottom left panel), and SFR (bottom right panel) using the four VMC overdensity classes (see Sect. \ref{sec:red_sequence} and Table \ref{tab:fraction_redseq}). For each class in each histogram, the leftmost and rightmost bins contain all objects with values respectively below and above the corresponding limits. (*fig:histograms1d*)

</div>
<div id="div_fig2">

<img src="tmp_2312.00924/./figures/arc_CGr32_map.png" alt="Fig17" width="100%"/>

**Figure 17. -** F814W HST-ACS image centered on the lensing source, 152\_CGr32, at $z=0.72519$ in logarithmic scale (arbitrary unit). The red contours represent the $\lya$ emission of the lensed source at $z=4.0973$ at levels 1, 2, and $4\times 10^{-17}$ erg s$^{-1}$ cm$^{-2}$ arcsec$^{-1}$.
 It is estimated by collapsing the MUSE cube over 20 Å at the observed wavelength of the $\lya$ line, and by subtracting the continuum using 50 Å wide adjacent wavelength ranges on both sides of the $\lya$ range. The four images are identified with black crosses, the lens model critical lines are displayed in purple,
 and the blue wedge indicates the range in the directions to the massive structure responsible for the shear in the lens model, at 1-$\sigma$. The green wedge represents the center directions between the barycenter of CGr32 members inferred from MUSE data and the X-ray center. (*fig:arc_map*)

</div>
<div id="div_fig3">

<img src="tmp_2312.00924/./figures/histo_mstar_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover_nored.png" alt="Fig22.1" width="50%"/><img src="tmp_2312.00924/./figures/histo_BD_ReffG_zmin0.25_zmax1.50_zppl15.0_zppu25.5_VMCover_nored.png" alt="Fig22.2" width="50%"/>

**Figure 22. -** Similar as Fig. \ref{fig:histograms1d} for the stellar mass and B/D, discarding quiescent galaxies. (*fig:histograms1d_nored*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.00924"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Separating the spectral counterparts in NGC 1275/Perseus cluster in X-rays

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.01174-b31b1b.svg)](https://arxiv.org/abs/2312.01174)<mark>Appeared on: 2023-12-05</mark> -  _18 pages, 6 pages, 8 tables_

</div>
<div id="authors">

E. Fedorova, et al. -- incl., <mark>N. Pulatova</mark>

</div>
<div id="abstract">

**Abstract:** We develop the recipe to separate the spectral counterparts of the AGN NGC 1275 from the emission of the Perseus cluster surrounding it in the spectra observed by Suzaku/XIS cameras with no usage of the spectral fitting models. The Perseus cluster emission reaches higher energies than is typical for the most AGN-situated dense surroundings (i.e. up to 9-10 keV). That is why the separation between the AGN and cluster spectra is especially important in this case. To avoid the degeneracy due to the huge quantity of the spectral fitting parameters such as abundances of elements the cluster consists of, thermal and Compton emission of the nucleus itself, and the jet SSC/IC emission spectral parameters as well we prefer to avoid the spectral fitting usage to perform this task. Instead, we use the spatial resolution of the components and double background subtracting. For this purpose we choose the following regions to collect all the photons from them: (1) circular or square-shaped region around the source (AGN); (2) ring-shaped (or non-overlapped square) region surrounding the AGN (for cluster); (3) remote empty circular region for the background. Having collected the photons from those regions we subtract the background (i.e. photons from the third region) from the source and cluster spectra. Next, we subtract the re-normalized cluster counts from the AGN spectrum; using the relation between the emission line amplitudes in the AGN and cluster spectra as the renormalization coefficient.We have performed this procedure on the whole set of the Suzaku/XIS observational data for NGC 1275 to obtain the cleaned spectra and light curve of the AGN emission in this system.

</div>

<div id="div_fig1">

<img src="tmp_2312.01174/./Screenshot_20231107_130040.png" alt="Fig4.1" width="50%"/><img src="tmp_2312.01174/./Screenshot_20231107_130713.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** The 1-10 keV range unfolded spectra of NGC 1275 by Suzaku/XIS for the observation performed on August 2006 (left) and August 2007 (right) with the models. (*unfspecs*)

</div>
<div id="div_fig2">

<img src="tmp_2312.01174/./lightcurve.png" alt="Fig5" width="100%"/>

**Figure 5. -** The 2-10 keV Suzaku/XIS light curve of NGC 1275. (*lcurve*)

</div>
<div id="div_fig3">

<img src="tmp_2312.01174/./spectral_lines1.png" alt="Fig1" width="100%"/>

**Figure 1. -** The example of the 0.5-10.0 keV spectra of the surroundings (cluster) and NGC 1275 (uncleaned AGN + cluster) extracted in the standard way for the observational window 101012010. The uncleaned AGN + cluster spectrum extracted from the circular central region around the AGN is shown in red, and the surrounding cluster spectrum extracted from the ring-shaped region is shown in black. (*spectrum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.01174"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tabularnewline}{\\}$
$\newcommand{\Mearth}{M_{\oplus}}$
$\newcommand{\Mjup}{M_{\text{Jup}}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\bPic}{\beta Pictoris}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\micron}{\unit{\micro\meter}}$
$\newcommand{\HD}{HD 181327}$
$\newcommand{\JM}[1]{\textcolor{magenta}{ [\textbf{JM: } #1]}}$
$\newcommand{\pt}[2]{\textcolor{red}{\textbf{PT:} #1}}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$</div>



<div id="title">

# The polarisation properties of the $\HD$  debris ring

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.02000-b31b1b.svg)](https://arxiv.org/abs/2312.02000)<mark>Appeared on: 2023-12-05</mark> -  _21 pages, 22 figures, accepted in Astronomy & Astrophysics_

</div>
<div id="authors">

J. Milli, et al. -- incl., <mark>J. Olofsson</mark>, <mark>C. Desgrange</mark>

</div>
<div id="abstract">

**Abstract:** Polarisation is a powerful remote-sensing tool to study the nature of particles scattering the starlight. It is widely used to characterise interplanetary dust particles in the Solar System and increasingly employed to investigate extrasolar dust in debris discs' systems. We aim to measure the scattering properties of the dust from the debris ring around $\HD$ at near-infrared wavelengths. We obtained high-contrast polarimetric images of $\HD$ in the H band with the SPHERE / IRDIS instrument on the Very Large Telescope (ESO). We complemented them with archival data from HST / NICMOS in the F110W filter reprocessed in the context of the Archival Legacy Investigations of Circumstellar Environments (ALICE) project. We developed a combined forward-modelling framework to simultaneously retrieve the scattering phase function in polarisation and intensity. We detected the debris disc around $\HD$ in polarised light and total intensity. We measured the scattering phase function and the degree of linear polarisation of the dust at 1.6 $\micron$ in the birth ring. The maximum polarisation is $23.6\% \pm 2.6\%$ and occurs between a scattering angle of $70^\circ$ and $82^\circ$ . We show that compact spherical particles made of a highly refractive and relatively absorbing material in a differential power-law size distribution of exponent $-3.5$ can simultaneously reproduce the polarimetric and total intensity scattering properties of the dust. This type of material cannot be obtained with a mixture of silicates, amorphous carbon, water ice, and porosity, and requires a more refracting component such as iron-bearing minerals. We reveal a striking analogy between the near-infrared polarisation of comets and that of $\HD$ . The methodology developed here combining VLT/SPHERE and HST/NICMOS may be applicable in the future to combine the polarimetric capabilities of SPHERE with the sensitivity of JWST.

</div>

<div id="div_fig1">

<img src="tmp_2312.02000/./figures/I_Qphi_Uphi.png" alt="Fig13" width="100%"/>

**Figure 13. -** Final images of the intensity (Stokes I, left image), azimuthal Stokes $Q_\phi$(middle) and $U_\phi$(right), calibrated in mJy/arcsec$^2$(linear colour scale). North is up and east to the left. The features in the south-west at $\sim2$\arcsec  separation are a cluster of bad pixels on the IRDIS detector. A numerical mask of inner radius 1.25$\arcsec$  and outer radius and 2.39$\arcsec$  was applied on the intensity image. (*fig_Qphi_Uphi*)

</div>
<div id="div_fig2">

<img src="tmp_2312.02000/./figures/optical_depth_ALMA_STIS_IRDIS_final.png" alt="Fig3" width="100%"/>

**Figure 3. -** Deprojected normalised optical depth of the disk at mm wavelengths \cite[orange line extracted from data published in][]{Pawellek2021}, in the optical with STIS \cite[green line][assuming a 2\% uncertainty]{Stark2014}, and in the near-infrared with IRDIS (blue line). The top panel shows the data (plain lines) with the $1  \sigma$ uncertainty (shade) and the double-power law fit (dotted line limited to the range of separation where the fit was performed). The bottom panel is a zoom in the region where the optical depth peaks, highlighting with vertical dotted line the location of the maxima. (*fig_tau*)

</div>
<div id="div_fig3">

<img src="tmp_2312.02000/./figures/NICMOS_best_phase_function.png" alt="Fig5" width="100%"/>

**Figure 5. -** SPF in intensity parametrising the dust scattered light measured by NICMOS the best. The 3 degrees of freedom of the SPF are shown as black dots, and the model performs a cubic interpolation (black curve).  (*fig_NICMOS_SPF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.02000"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

333  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
